# MP Neuron

In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import re

In [9]:
train = pd.read_csv(r"D:\git\PRACTICE\DATA\train.csv")
test = pd.read_csv(r"D:\git\PRACTICE\DATA\test.csv")

In [10]:
features = ["RAM", "Capacity", "Pixel Density", "Internal Memory"]
target = "Rating"

In [25]:
# Function to extract numbers from text
def extract_number(value):
    if pd.isnull(value):
        return np.nan
    value = str(value)

    match = re.search(r'\d+', value.replace(',', ''))
    if match:
        return float(match.group())
    return np.nan



In [ ]:
for col in features:
    train[col] = train[col].apply(extract_number)
    test[col] = test[col].apply(extract_number)


In [ ]:
for col in features:
    train[col] = train[col].fillna(train[col].median())
    test[col] = test[col].fillna(train[col].median())
    
  # use train median

In [15]:
train["target"] = (train["Rating"] > 4.5).astype(int)

In [16]:
X_train = train[features].values
y_train = train["target"].values
X_test = test[features].values

In [17]:
X_train = X_train / X_train.max(axis=0)
X_test = X_test / X_train.max(axis=0)

In [18]:
class MPNeuron:
    def __init__(self, input_size, threshold=0.5):
        self.weights = np.random.rand(input_size)
        self.threshold = threshold

    def predict(self, X):
        linear_output = np.dot(X, self.weights)
        return (linear_output > self.threshold).astype(int)

mp_neuron = MPNeuron(X_train.shape[1])

In [19]:
# Predictions
y_pred_train = mp_neuron.predict(X_train)
y_pred_test = mp_neuron.predict(X_test)

In [20]:
train_acc = np.mean(y_pred_train == y_train)
print("MP Neuron Train Accuracy:", train_acc)

MP Neuron Train Accuracy: 0.9408450704225352


In [21]:
print("MP Neuron Test Predictions (first 10 rows):", y_pred_test[:10])

MP Neuron Test Predictions (first 10 rows): [1 1 1 1 1 1 1 1 1 1]


In [ ]:
'''output = pd.DataFrame({
    "PhoneId": test["PhoneId"],
    "Class": y_pred_test
})
output.to_excel(r"D:\git\PRACTICE\DATA\OUTPUT\mp_neuron_test_predictions.xlsx", index=False)
print("Test predictions saved to mp_neuron_test_predictions.xlsx")'''

# final 
# Random weights
# Fixed threshold
# No learning from y_train

# PERCEPTRON

In [ ]:
class Perceptron:
    def __init__(self, input_size, lr=0.01, epochs=1000):
        self.weights = np.zeros(input_size)
        self.bias = 0
        self.lr = lr
        self.epochs = epochs

    def activation(self, x):
        return np.where(x > 0, 1, 0)

    def fit(self, X, y):
        for _ in range(self.epochs):
            for xi, target in zip(X, y):
                linear_output = np.dot(xi, self.weights) + self.bias
                y_pred = self.activation(linear_output)
                error = target - y_pred
                # Update rule
                self.weights += self.lr * error * xi
                self.bias += self.lr * error

    def predict(self, X):
        linear_output = np.dot(X, self.weights) + self.bias
        return self.activation(linear_output)
    

# Starts with zero weights

# Updates weights using training data

# Learns gradually → train accuracy improves

In [27]:
perceptron = Perceptron(X_train.shape[1], lr=0.01, epochs=1000)
perceptron.fit(X_train, y_train)

y_pred_train = perceptron.predict(X_train)
y_pred_test = perceptron.predict(X_test)

In [28]:
train_acc = np.mean(y_pred_train == y_train)
print("Perceptron Train Accuracy:", train_acc)


Perceptron Train Accuracy: 0.9577464788732394


In [ ]:
'''print("Perceptron Test Predictions (first 10 rows):", y_pred_test[:10])
output = pd.DataFrame({
    "PhoneId": test["PhoneId"],
    "Class": y_pred_test
})
output.to_excel(r"D:\git\PRACTICE\DATA\OUTPUT\perceptron_test_predictions.xlsx", index=False)
print("Test predictions saved to perceptron_test_predictions.xlsx")'''

Perceptron Test Predictions (first 10 rows): [1 1 1 1 1 1 1 1 1 1]
Test predictions saved to perceptron_test_predictions.xlsx


# SIGMOID

In [35]:
features = [
    "RAM", "Capacity", "Pixel Density", "Internal Memory",
    "Height", "Width", "Screen Size", "Weight"
]

In [53]:
def extract_number(series):
    # Extract numbers from strings and convert to float
    return series.astype(str).str.extract(r'([\d.]+)')[0].astype(float)

for col in ["Capacity", "Internal Memory", "Screen Size", "Weight"]:
    train[col] = extract_number(train[col])
    test[col] = extract_number(test[col])

In [54]:
for col in features:
    train[col] = train[col].fillna(train[col].median())
    test[col] = test[col].fillna(train[col].median())


c:\Users\ERS1397\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\ERS1397\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [55]:

train["target"] = (train["Rating"] > 4.5).astype(int)


In [56]:
train_majority = train[train.target == 0]
train_minority = train[train.target == 1]

train_minority_upsampled = resample(train_minority,
                                    replace=True,
                                    n_samples=len(train_majority),
                                    random_state=42)

train_balanced = pd.concat([train_majority, train_minority_upsampled])
X_train = train_balanced[features].values
y_train = train_balanced["target"].values
X_test = test[features].values

In [61]:
X_train = X_train / X_train.max(axis=0)
X_test = X_test / X_train.max(axis=0)

In [62]:
class SigmoidNeuron:
    def __init__(self, input_size, lr=0.1, epochs=1000):
        self.weights = np.random.rand(input_size)
        self.bias = 0
        self.lr = lr
        self.epochs = epochs

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        for _ in range(self.epochs):
            z = np.dot(X, self.weights) + self.bias
            y_pred = self.sigmoid(z)
            error = y - y_pred
            # Gradient descent
            self.weights += self.lr * np.dot(X.T, error)
            self.bias += self.lr * error.mean()

    def predict_proba(self, X):
        z = np.dot(X, self.weights) + self.bias
        return self.sigmoid(z)

    def predict(self, X, threshold=0.5):
        probs = self.predict_proba(X)
        return (probs > threshold).astype(int)

In [ ]:
sig_neuron = SigmoidNeuron(X_train.shape[1], lr=0.1, epochs=1000)
sig_neuron.fit(X_train, y_train)

y_prob_train = sig_neuron.predict_proba(X_train)
y_pred_train = sig_neuron.predict(X_train, threshold=0.9)
y_prob_test = sig_neuron.predict_proba(X_test)
y_pred_test = sig_neuron.predict(X_test, threshold=0.9)

In [71]:
train_acc = np.mean(y_pred_train == y_train)
print("Sigmoid Neuron Train Accuracy:", train_acc)


Sigmoid Neuron Train Accuracy: 0.5


In [72]:
output = pd.DataFrame({
    "PhoneId": test["PhoneId"],
    "Probability": y_prob_test,
    "Class": y_pred_test
})
output.to_excel(r"D:\git\PRACTICE\DATA\OUTPUT\sigmoid_test_predictions.xlsx", index=False)
print("Test predictions saved to sigmoid_neuron_test_predictions.xlsx")

Test predictions saved to sigmoid_neuron_test_predictions.xlsx
